In [ ]:
from efaar_benchmarking.data_loading import load_periscope
from efaar_benchmarking.efaar import *
from efaar_benchmarking.constants import *
from efaar_benchmarking.benchmarking import benchmark, univariate_consistency_benchmark
from efaar_benchmarking.plotting import plot_recall

recall_threshold_pairs = []
start = 0.01
end = 0.99
step = 0.01

while start <= .105 and end >= .895:
    recall_threshold_pairs.append((round(start,2), round(end,2)))
    start += step
    end -= step

print(recall_threshold_pairs)

In [ ]:
ct = "A549"
features, metadata = load_periscope(cell_type=ct, normalized=False)
embeddings = embed_align_by_pca(features.values, metadata, variance_or_ncomp=None, plate_col=PERISCOPE_PLATE_COL)
embeddings = align_on_controls(embeddings, metadata, pert_col=PERISCOPE_PERT_LABEL_COL, control_key=PERISCOPE_CONTROL_PERT_LABEL, scale=True)
phenoprint_pvals = univariate_consistency_benchmark(embeddings, metadata, pert_col=PERISCOPE_PERT_LABEL_COL, keys_to_drop=[PERISCOPE_CONTROL_PERT_LABEL, 'negCtrl'])
map_data = aggregate(embeddings, metadata, pert_col=PERISCOPE_PERT_LABEL_COL, control_key=PERISCOPE_CONTROL_PERT_LABEL)
metrics = benchmark(map_data, recall_thr_pairs=recall_threshold_pairs, pert_col=PERISCOPE_PERT_LABEL_COL)
plot_recall(metrics)
metrics.groupby('source')['recall_0.05_0.95'].mean()